In [1]:
import os
import zipfile

try:
    from google.colab import drive
    RUNNING_IN_COLAB = True
    
except ImportError:
    drive = None
    RUNNING_IN_COLAB = False

if RUNNING_IN_COLAB:
    os.system('git clone https://github.com/vekshinnikita/face_recognition.git /content/face_recognition')
    os.chdir('/content/face_recognition') 
    
    drive.mount('/content/drive')
    
    zip_dataset_path = '/content/drive/MyDrive/recognition_dataset.zip'
    destination_dataset_path = '/content/face_recognition/'
    with zipfile.ZipFile(zip_dataset_path, 'r') as zip_ref:
      zip_ref.extractall(destination_dataset_path)

In [2]:
import torch
from ultralytics import YOLO
from utils.system import get_available_device
from facenet_pytorch import InceptionResnetV1
import torch.nn.functional as F
import numpy as np
import faiss


device = get_available_device()

KNOWN_FACES_PATH='./known_faces'

# Model
detector_model = YOLO("./best_models/yolo_decoder_best.pt", verbose=False).to(device)
classifier_model = InceptionResnetV1(pretrained='vggface2', num_classes=2, device=device).eval()

/Users/user/Documents/university/Дипломы/Никита/project/venv_yolo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU is not available, using CPU instead
Using device: cpu


In [3]:
known_embeddings = list()
class_embedding_names = list()

index = faiss.IndexFlatL2(512)

In [4]:
from torchvision.transforms import v2

transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.Resize((160, 160)),  # Измените размер под вашу модель
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])


def compare_embeddings(embedding, threshold=0.7):
    # 3. Поиск ближайшего embedding:
    embedding_np = embedding.detach().numpy()
    embedding_np = embedding_np.reshape(1, -1).astype('float32')  # Убедитесь, что форма и тип данных верны

    distances, indices = index.search(embedding_np, k=1)
    if distances > threshold:
        return None
    
    # 4. Получение имени объекта по индексу:
    closest_object_name = class_embedding_names[indices[0][0]]  # indices[0][0] - индекс ближайшего embedding

    return closest_object_name
    

def detect_faces(image):
    with torch.no_grad():  # Отключите вычисление градиентов во время инференса
        result = detector_model([image], verbose=False)[0]
    
    return result.boxes.xyxy
    
        
def recognize_faces(image, boxes):
    cropped_faces = [
        transform(image.crop(list(map(float,bbox))))
        for bbox in boxes
    ]
    if cropped_faces:  # Убедитесь, что список не пустой
        face_tensors = torch.stack(cropped_faces).to(device)
        embeddings = classifier_model(face_tensors)
    else:
        embeddings = torch.empty(0).to(device)
        
    return embeddings

def recognize_cropped_face(image):
    tensor = transform(image).unsqueeze(0).to(device)
    embedding = classifier_model(tensor)[0]
    
    return embedding
    

In [5]:
from dataset_classes.vgg_face2_face_recognition import VGGFace2FaceRecognitionDataset
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import torch


def reverse_transform(image: torch.Tensor):
    fn = v2.Compose([
        v2.Normalize(mean=(-0.5 / 0.5, -0.5 / 0.5, -0.5 / 0.5), std=(1 / 0.5, 1 / 0.5, 1 / 0.5)),
    ])
    
    return fn(image)

dataset = VGGFace2FaceRecognitionDataset(
    './recognition_dataset/images/test',
    './recognition_dataset/labels/test',
)
known_face_map = dataset.known_face_map
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [6]:
for class_name, path_list in known_face_map.items():
    for file_path in path_list:
        face_image = dataset._get_face_image(file_path)
        embedding = recognize_cropped_face(face_image)
        
        known_embeddings.append(embedding)
        class_embedding_names.append(class_name)
        
known_embeddings_detached = [tensor.detach().numpy() for tensor in known_embeddings] # отсоединяем каждый Tensor
known_embeddings = np.array(known_embeddings_detached).astype('float32')
index.add(known_embeddings)

In [7]:
from tqdm import tqdm

test_loop = tqdm(dataset)
total = 0
tp = 0
tn = 0
fp = 0
fn = 0

for image, label in test_loop:
    total += 1
    boxes = detect_faces(image)
    embeddings = recognize_faces(image, boxes)

    for bbox, embedding in zip(boxes, embeddings):
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])
        
        predicted_label = compare_embeddings(embedding)
        if predicted_label == label:
            tp += 1
        elif predicted_label != label and predicted_label is not None:
            fp += 1
        elif predicted_label is None:
            fn += 1
    
    if total > 10000:
        break
            
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1 = 2 * (precision * recall) / (precision + recall)
accuracy = (tp + tn) / total

print('Accuracy: ', accuracy)
print('FAR: ', fp/total)
print('FRR: ', fn/total)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1-Score: ', f1)

  6%|▌         | 10000/167876 [23:29<6:10:45,  7.10it/s]

Accuracy:  0.635036496350365
FAR:  0.025197480251974803
FRR:  0.34206579342065796
Precision:  0.9618355293048614
Recall:  0.6499181334424887
F1-Score:  0.7756946564885496
